In [14]:
import numpy as np
import pandas as pd 
import json
from collections import defaultdict
import copy
import pickle 
import gensim
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [57]:
import multiprocessing
from gensim.models import Word2Vec,KeyedVectors

In [19]:
cores = multiprocessing.cpu_count()
cores

10

In [21]:
class MySentences(object):
    def __init__(self, filename):
        self.file=filename
 
    def __iter__(self):
        f = open(self.file)
        for line in f:
            yield line.split()
            
sentences = MySentences('data/word2vec_inputs/abstracts_k40_frex0.5.txt') # a memory-friendly iterator

In [91]:
w2v_model = Word2Vec(sg=1, vector_size=100,window=5, workers=cores-2, 
                               epochs=100, min_count=5, max_vocab_size=None)

INFO - 17:59:13: Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=100, alpha=0.025)', 'datetime': '2022-03-19T17:59:13.402684', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:24:02) \n[Clang 11.1.0 ]', 'platform': 'macOS-12.1-arm64-arm-64bit', 'event': 'created'}


In [92]:
w2v_model.build_vocab(sentences, progress_per=10000)

INFO - 17:59:14: collecting all words and their counts
INFO - 17:59:14: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 17:59:14: PROGRESS: at sentence #10000, processed 584051 words, keeping 10915 word types
INFO - 17:59:14: PROGRESS: at sentence #20000, processed 1170414 words, keeping 13372 word types
INFO - 17:59:14: PROGRESS: at sentence #30000, processed 1757163 words, keeping 14845 word types
INFO - 17:59:14: PROGRESS: at sentence #40000, processed 2339028 words, keeping 15771 word types
INFO - 17:59:14: PROGRESS: at sentence #50000, processed 2917972 words, keeping 16447 word types
INFO - 17:59:14: PROGRESS: at sentence #60000, processed 3504117 words, keeping 16955 word types
INFO - 17:59:14: PROGRESS: at sentence #70000, processed 4082316 words, keeping 17358 word types
INFO - 17:59:14: PROGRESS: at sentence #80000, processed 4663542 words, keeping 17650 word types
INFO - 17:59:15: PROGRESS: at sentence #90000, processed 5243428 words, keeping 17912 w

In [93]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)

INFO - 17:59:20: Word2Vec lifecycle event {'msg': 'training model with 8 workers on 17933 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2022-03-19T17:59:20.922477', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:24:02) \n[Clang 11.1.0 ]', 'platform': 'macOS-12.1-arm64-arm-64bit', 'event': 'train'}
INFO - 17:59:21: EPOCH 1 - PROGRESS: at 9.63% examples, 1299456 words/s, in_qsize 16, out_qsize 0
INFO - 17:59:22: EPOCH 1 - PROGRESS: at 19.93% examples, 1346380 words/s, in_qsize 15, out_qsize 0
INFO - 17:59:23: EPOCH 1 - PROGRESS: at 30.11% examples, 1355347 words/s, in_qsize 16, out_qsize 0
INFO - 17:59:24: EPOCH 1 - PROGRESS: at 40.28% examples, 1359937 words/s, in_qsize 15, out_qsize 0
INFO - 17:59:25: EPOCH 1 - PROGRESS: at 50.32% examples, 1357676 words/s, in_qsize 15, out_qsize 0
INFO - 17:59:26: EPOCH 1 - PROGRESS: at 60.15% examples, 1351516 words/s, in_qsize 15, out_qsize 

INFO - 18:00:11: worker thread finished; awaiting finish of 6 more threads
INFO - 18:00:11: worker thread finished; awaiting finish of 5 more threads
INFO - 18:00:11: worker thread finished; awaiting finish of 4 more threads
INFO - 18:00:11: worker thread finished; awaiting finish of 3 more threads
INFO - 18:00:11: worker thread finished; awaiting finish of 2 more threads
INFO - 18:00:11: worker thread finished; awaiting finish of 1 more threads
INFO - 18:00:11: worker thread finished; awaiting finish of 0 more threads
INFO - 18:00:11: EPOCH - 5 : training on 14482753 raw words (13511686 effective words) took 10.2s, 1320152 effective words/s
INFO - 18:00:12: EPOCH 6 - PROGRESS: at 9.08% examples, 1232834 words/s, in_qsize 15, out_qsize 0
INFO - 18:00:13: EPOCH 6 - PROGRESS: at 19.32% examples, 1307925 words/s, in_qsize 15, out_qsize 0
INFO - 18:00:14: EPOCH 6 - PROGRESS: at 29.57% examples, 1331999 words/s, in_qsize 15, out_qsize 0
INFO - 18:00:15: EPOCH 6 - PROGRESS: at 39.93% example

INFO - 18:01:01: worker thread finished; awaiting finish of 4 more threads
INFO - 18:01:01: worker thread finished; awaiting finish of 3 more threads
INFO - 18:01:01: worker thread finished; awaiting finish of 2 more threads
INFO - 18:01:01: worker thread finished; awaiting finish of 1 more threads
INFO - 18:01:01: worker thread finished; awaiting finish of 0 more threads
INFO - 18:01:01: EPOCH - 10 : training on 14482753 raw words (13510557 effective words) took 9.8s, 1376109 effective words/s
INFO - 18:01:02: EPOCH 11 - PROGRESS: at 9.42% examples, 1282339 words/s, in_qsize 15, out_qsize 0
INFO - 18:01:03: EPOCH 11 - PROGRESS: at 18.76% examples, 1264852 words/s, in_qsize 14, out_qsize 2
INFO - 18:01:04: EPOCH 11 - PROGRESS: at 28.40% examples, 1276414 words/s, in_qsize 15, out_qsize 0
INFO - 18:01:05: EPOCH 11 - PROGRESS: at 38.23% examples, 1283618 words/s, in_qsize 15, out_qsize 0
INFO - 18:01:06: EPOCH 11 - PROGRESS: at 47.11% examples, 1266505 words/s, in_qsize 14, out_qsize 1
I

INFO - 18:01:52: worker thread finished; awaiting finish of 7 more threads
INFO - 18:01:52: worker thread finished; awaiting finish of 6 more threads
INFO - 18:01:52: worker thread finished; awaiting finish of 5 more threads
INFO - 18:01:52: worker thread finished; awaiting finish of 4 more threads
INFO - 18:01:52: worker thread finished; awaiting finish of 3 more threads
INFO - 18:01:52: worker thread finished; awaiting finish of 2 more threads
INFO - 18:01:52: worker thread finished; awaiting finish of 1 more threads
INFO - 18:01:52: worker thread finished; awaiting finish of 0 more threads
INFO - 18:01:52: EPOCH - 15 : training on 14482753 raw words (13511289 effective words) took 10.5s, 1286582 effective words/s
INFO - 18:01:53: EPOCH 16 - PROGRESS: at 8.06% examples, 1095092 words/s, in_qsize 15, out_qsize 0
INFO - 18:01:55: EPOCH 16 - PROGRESS: at 17.44% examples, 1177876 words/s, in_qsize 15, out_qsize 0
INFO - 18:01:56: EPOCH 16 - PROGRESS: at 27.77% examples, 1248335 words/s, 

INFO - 18:02:41: EPOCH 20 - PROGRESS: at 70.06% examples, 1349584 words/s, in_qsize 15, out_qsize 0
INFO - 18:02:42: EPOCH 20 - PROGRESS: at 79.58% examples, 1341010 words/s, in_qsize 15, out_qsize 0
INFO - 18:02:43: EPOCH 20 - PROGRESS: at 89.89% examples, 1346009 words/s, in_qsize 15, out_qsize 0
INFO - 18:02:44: worker thread finished; awaiting finish of 7 more threads
INFO - 18:02:44: worker thread finished; awaiting finish of 6 more threads
INFO - 18:02:44: worker thread finished; awaiting finish of 5 more threads
INFO - 18:02:44: worker thread finished; awaiting finish of 4 more threads
INFO - 18:02:44: worker thread finished; awaiting finish of 3 more threads
INFO - 18:02:44: worker thread finished; awaiting finish of 2 more threads
INFO - 18:02:44: worker thread finished; awaiting finish of 1 more threads
INFO - 18:02:44: worker thread finished; awaiting finish of 0 more threads
INFO - 18:02:44: EPOCH - 20 : training on 14482753 raw words (13514234 effective words) took 10.0s, 

...

INFO - 18:13:28: EPOCH 84 - PROGRESS: at 23.75% examples, 1051372 words/s, in_qsize 14, out_qsize 1
INFO - 18:13:29: EPOCH 84 - PROGRESS: at 32.51% examples, 1081491 words/s, in_qsize 16, out_qsize 0
INFO - 18:13:30: EPOCH 84 - PROGRESS: at 40.76% examples, 1085547 words/s, in_qsize 14, out_qsize 1
INFO - 18:13:31: EPOCH 84 - PROGRESS: at 49.09% examples, 1091446 words/s, in_qsize 15, out_qsize 0
INFO - 18:13:32: EPOCH 84 - PROGRESS: at 56.38% examples, 1068364 words/s, in_qsize 12, out_qsize 4
INFO - 18:13:33: EPOCH 84 - PROGRESS: at 64.48% examples, 1071464 words/s, in_qsize 16, out_qsize 0
INFO - 18:13:34: EPOCH 84 - PROGRESS: at 74.51% examples, 1102581 words/s, in_qsize 13, out_qsize 0
INFO - 18:13:35: EPOCH 84 - PROGRESS: at 84.69% examples, 1126615 words/s, in_qsize 16, out_qsize 0
INFO - 18:13:36: EPOCH 84 - PROGRESS: at 94.72% examples, 1146593 words/s, in_qsize 16, out_qsize 0
INFO - 18:13:36: worker thread finished; awaiting finish of 7 more threads
INFO - 18:13:36: worker t

INFO - 18:14:19: worker thread finished; awaiting finish of 1 more threads
INFO - 18:14:19: worker thread finished; awaiting finish of 0 more threads
INFO - 18:14:19: EPOCH - 88 : training on 14482753 raw words (13509719 effective words) took 10.5s, 1285581 effective words/s
INFO - 18:14:20: EPOCH 89 - PROGRESS: at 7.52% examples, 1022385 words/s, in_qsize 15, out_qsize 0
INFO - 18:14:21: EPOCH 89 - PROGRESS: at 17.65% examples, 1194266 words/s, in_qsize 16, out_qsize 0
INFO - 18:14:22: EPOCH 89 - PROGRESS: at 27.49% examples, 1237830 words/s, in_qsize 15, out_qsize 0
INFO - 18:14:23: EPOCH 89 - PROGRESS: at 35.27% examples, 1188383 words/s, in_qsize 14, out_qsize 1
INFO - 18:14:24: EPOCH 89 - PROGRESS: at 43.78% examples, 1178806 words/s, in_qsize 15, out_qsize 0
INFO - 18:14:25: EPOCH 89 - PROGRESS: at 52.04% examples, 1167431 words/s, in_qsize 15, out_qsize 0
INFO - 18:14:26: EPOCH 89 - PROGRESS: at 59.40% examples, 1140355 words/s, in_qsize 12, out_qsize 3
INFO - 18:14:27: EPOCH 89

INFO - 18:15:13: EPOCH 93 - PROGRESS: at 76.83% examples, 1025633 words/s, in_qsize 15, out_qsize 0
INFO - 18:15:14: EPOCH 93 - PROGRESS: at 83.86% examples, 1017771 words/s, in_qsize 15, out_qsize 0
INFO - 18:15:15: EPOCH 93 - PROGRESS: at 93.38% examples, 1040171 words/s, in_qsize 15, out_qsize 0
INFO - 18:15:15: worker thread finished; awaiting finish of 7 more threads
INFO - 18:15:15: worker thread finished; awaiting finish of 6 more threads
INFO - 18:15:15: worker thread finished; awaiting finish of 5 more threads
INFO - 18:15:15: worker thread finished; awaiting finish of 4 more threads
INFO - 18:15:15: worker thread finished; awaiting finish of 3 more threads
INFO - 18:15:15: worker thread finished; awaiting finish of 2 more threads
INFO - 18:15:15: worker thread finished; awaiting finish of 1 more threads
INFO - 18:15:15: worker thread finished; awaiting finish of 0 more threads
INFO - 18:15:15: EPOCH - 93 : training on 14482753 raw words (13511071 effective words) took 12.8s, 

INFO - 18:16:02: EPOCH 98 - PROGRESS: at 47.37% examples, 1272642 words/s, in_qsize 15, out_qsize 0
INFO - 18:16:03: EPOCH 98 - PROGRESS: at 57.00% examples, 1277394 words/s, in_qsize 16, out_qsize 0
INFO - 18:16:04: EPOCH 98 - PROGRESS: at 67.38% examples, 1295316 words/s, in_qsize 15, out_qsize 0
INFO - 18:16:05: EPOCH 98 - PROGRESS: at 77.31% examples, 1301615 words/s, in_qsize 16, out_qsize 0
INFO - 18:16:06: EPOCH 98 - PROGRESS: at 87.58% examples, 1310738 words/s, in_qsize 15, out_qsize 0
INFO - 18:16:07: EPOCH 98 - PROGRESS: at 97.84% examples, 1316491 words/s, in_qsize 15, out_qsize 0
INFO - 18:16:07: worker thread finished; awaiting finish of 7 more threads
INFO - 18:16:07: worker thread finished; awaiting finish of 6 more threads
INFO - 18:16:07: worker thread finished; awaiting finish of 5 more threads
INFO - 18:16:07: worker thread finished; awaiting finish of 4 more threads
INFO - 18:16:07: worker thread finished; awaiting finish of 3 more threads
INFO - 18:16:07: worker t

(1351143349, 1448275300)

In [94]:
w2v_model.wv.most_similar(positive=["laser"])

[('puls', 0.788336455821991),
 ('tisapphir', 0.7467207312583923),
 ('wcm', 0.7307332754135132),
 ('ultrashort', 0.7296327352523804),
 ('high-intens', 0.7285068035125732),
 ('pump', 0.7099785208702087),
 ('-fs', 0.6789311766624451),
 ('xuv', 0.6740944385528564),
 ('short-puls', 0.6731037497520447),
 ('frequency-doubl', 0.6634376645088196)]

In [95]:
w2v_model.wv.most_similar(positive=["black"])

[('black-hol', 0.8745536804199219),
 ('schwarzschild', 0.8461962342262268),
 ('horizon', 0.8079009056091309),
 ('reissner-nordström', 0.8078852891921997),
 ('kerr-newman', 0.7697179317474365),
 ('spacetim', 0.7551360130310059),
 ('wormhol', 0.754063606262207),
 ('myers-perri', 0.7506564855575562),
 ('hole', 0.7470870018005371),
 ('near-horizon', 0.7397699356079102)]

In [96]:
w2v_model.wv.most_similar(positive=["magnet"])

[('ferromagnet', 0.6860878467559814),
 ('koe', 0.6853837966918945),
 ('magnetic-field', 0.6658287644386292),
 ('antiferromagnet', 0.664893627166748),
 ('cant', 0.6495588421821594),
 ('nonmagnet', 0.6474876999855042),
 ('ferrimagnet', 0.633202075958252),
 ('spin-flop', 0.622723400592804),
 ('paramagnet', 0.6144354343414307),
 ('μbco', 0.6113200783729553)]

In [97]:
w2v_model.wv.most_similar(positive=["bandgap"])

[('band-gap', 0.7489559650421143),
 ('conduction-band', 0.5912713408470154),
 ('band', 0.5665439367294312),
 ('gan', 0.5478387475013733),
 ('heterojunct', 0.5440356731414795),
 ('gap', 0.540385365486145),
 ('heterostructur', 0.5383453965187073),
 ('alxga−xn', 0.5379083752632141),
 ('wurtzit', 0.533138632774353),
 ('valence-band', 0.5288936495780945)]

In [98]:
w2v_model.wv.most_similar(positive=["quantum"])

[('qubit', 0.6023014187812805),
 ('“quantum', 0.5993359684944153),
 ('classic', 0.5982183218002319),
 ('decoher', 0.540327787399292),
 ('quantum-inform', 0.5328150987625122),
 ('continuous-vari', 0.5260676145553589),
 ('quantum-class', 0.5127130746841431),
 ('many-particl', 0.5107666850090027),
 ('quantum-mechan', 0.5036872625350952),
 ('dick', 0.49693864583969116)]

In [99]:
w2v_model.wv.most_similar(positive=["thermal"])

[('heat', 0.6052266359329224),
 ('nontherm', 0.6018601059913635),
 ('equilibrium', 0.5247296094894409),
 ('nonequilibrium', 0.5236335396766663),
 ('cross-plan', 0.5184169411659241),
 ('joul', 0.5026931166648865),
 ('temperatur', 0.49233219027519226),
 ('ballist', 0.48988279700279236),
 ('hot', 0.48499634861946106),
 ('transport', 0.4838297367095947)]

In [100]:
w2v_model.wv.most_similar(positive=["light"])

[('illumin', 0.5874162316322327),
 ('monochromat', 0.5279420018196106),
 ('shed', 0.5266191959381104),
 ('mirror', 0.5147891640663147),
 ('high-intens', 0.5031155347824097),
 ('photon', 0.5019097328186035),
 ('radiat', 0.49534285068511963),
 ('polar', 0.47170019149780273),
 ('laser', 0.4714818596839905),
 ('heavi', 0.47124093770980835)]

In [108]:
w2v_model.wv.most_similar(negative=["atom"])

[('doihttpsdoiorgphysrevd', 0.22202488780021667),
 ('flow-align', 0.16191434860229492),
 ('chiral-glass', 0.15969425439834595),
 ('deem', 0.1509760469198227),
 ('witten-veneziano', 0.13666196167469025),
 ('hawking-pag', 0.1332055777311325),
 ('sgs', 0.12676627933979034),
 ('kogan', 0.1255721151828766),
 ('spreader', 0.12343037873506546),
 ('rate--strain', 0.11967514455318451)]

In [101]:
w2v_model.wv.similarity("black", 'hole')

0.747087

In [102]:
w2v_model.wv.similarity("laser", 'hole')

0.12577716

In [118]:
w2v_model.wv.similarity("laser", 'beam')

0.64177763

In [119]:
w2v_model.wv.similarity("bandgap", 'semiconductor')

0.5144048

In [110]:
w2v_model.wv.most_similar(positive=["acceler", "black"], negative=["deceler"], topn=3)

[('black-hol', 0.7066452503204346),
 ('schwarzschild', 0.6604212522506714),
 ('nake', 0.6242992877960205)]

# Save model

In [111]:
# save 
w2v_bin_path = 'data/word2vec_outputs/model_word2vec_format_k20_frex0.25.bin'
w2v_model.wv.save_word2vec_format(w2v_bin_path,binary=True)

INFO - 18:24:48: storing 17933x100 projection weights into data/word2vec_outputs/model_word2vec_format_k20_frex0.25.bin


# Load model

In [112]:
# load
loaded_model = KeyedVectors.load_word2vec_format(w2v_bin_path, binary=True)

INFO - 18:24:50: loading projection weights from data/word2vec_outputs/model_word2vec_format_k20_frex0.25.bin
INFO - 18:24:50: KeyedVectors lifecycle event {'msg': 'loaded (17933, 100) matrix of type float32 from data/word2vec_outputs/model_word2vec_format_k20_frex0.25.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-03-19T18:24:50.138063', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:24:02) \n[Clang 11.1.0 ]', 'platform': 'macOS-12.1-arm64-arm-64bit', 'event': 'load_word2vec_format'}


In [120]:
loaded_model.most_similar(positive=["thermal"])

[('heat', 0.6052266359329224),
 ('nontherm', 0.6018601059913635),
 ('equilibrium', 0.5247296094894409),
 ('nonequilibrium', 0.5236335396766663),
 ('cross-plan', 0.5184169411659241),
 ('joul', 0.5026931166648865),
 ('temperatur', 0.49233219027519226),
 ('ballist', 0.48988279700279236),
 ('hot', 0.48499634861946106),
 ('transport', 0.4838297367095947)]

In [113]:
loaded_model.similarity("laser", 'beam')

0.64177763

In [114]:
loaded_model.most_similar(positive=["acceler", "black"], negative=["deceler"], topn=3)

[('black-hol', 0.7066452503204346),
 ('schwarzschild', 0.6604212522506714),
 ('nake', 0.6242992877960205)]